In [2]:
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, Sequential, MessagePassing
from torch_geometric.utils import add_self_loops
import numpy as np
from torch_geometric.data import Batch
import torch


torch.backends.cudnn.benchmark = True
DEVICE = torch.device("cuda:0" if not torch.cuda.is_available() else "cpu") 

In [155]:
class TemporalLayer(torch.nn.Module):
    """
    Contains non-linear activation function ReLu.
    This likely wont be an effective approximation for the data.
    You could experiment with sigmoid, or preferably,
    you could try using a non-linear layer such as:
        - a multi-layer perceptron (MLP)
        - RNN with
            - a gated recurrent unit (GRU)
            - long short-term memory (LSTM) unit
        - RNN layer can add significant performance see:
            https://ieeexplore.ieee.org/document/9649108
    """
    def __init__(self, in_channels, out_channels):
        super(TemporalLayer, self).__init__()
        self.linear = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index, edge_attr):
        # Sort the edges by timestamp to ensure the correct temporal order
        #edge_attr, perm = edge_attr.sort()
        #edge_index = edge_index[:, perm]
        #x = x[perm]

        # Aggregate the temporal information for each node using the mean
        node_attrs = torch.zeros(x.shape[0], edge_attr.shape[1], device=x.device)
        node_attrs.index_add_(0, edge_index[1], edge_attr)
        node_counts = torch.zeros(x.shape[0], edge_attr.shape[1], device=x.device)
        node_counts.index_add_(0, edge_index[1], torch.ones_like(edge_attr))
        node_attrs /= node_counts

        # Pass the node attributes through a linear layer to get the temporal embeddings
        x = F.relu(self.linear(node_attrs))

        return x

class GCNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(1, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.temporal = TemporalLayer(in_channels, hidden_channels)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
        print(edge_weight.shape, edge_index.shape, x.shape)
        
        
        
        #x = self.temporal(x, edge_index, edge_attr)  # Pass through temporal layer
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [7]:
def train(train_loader, net, LR=0.1, epochs=2000, val_loader=None):
    net.to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr=LR)
    criterion = nn.L1Loss()
    all_MSE = nn.L1Loss()
    val_losses = []
    print(f"Using: {DEVICE}")
                            
    parameter_loss = []
    losses = []
    processed = 0
    last_loss = 0
    for epoch in range(epochs):
        loss = 0
        
        net.train()
        with tqdm(train_loader, unit="batch") as it:
            if epoch > 0:
                it.set_postfix(lastLoss=last_loss, valLoss=val_losses[-1])
            for idx, batch in enumerate(it):
                it.set_description(f"Epoch {epoch+1}/{epochs}")
                batch.to(DEVICE)
                
                optimizer.zero_grad()
                out = model(batch)
                loss = loss_fn(out, batch.y)
                loss.backward()
                optimizer.step()
                
                for i in range(len(predicted)):
                    current_MSE = []
                    for j in range(6):
                        current_MSE.append(all_MSE(out[i][j], predicted[i][j]).item())
                    parameter_loss.append(current_MSE)
                    processed += 1
        
        if val_loader:
            val_loss = 0
            net.eval()
            for idx, data in enumerate(val_loader):
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)

                predicted = net(inp)
                cost = criterion(out, predicted)
                val_loss += cost.item()
            val_loss /= len(val_loader)  
            val_losses.append(val_loss)
        
        losses.append(loss)
        last_loss = loss/len(it)
    print("Parameters: Skin YM, Adipose YM, Skin PR, Adipose PR, Skin Perm, Adipose Perm")
    print(f"Sampled Ranges: 10e3 - 50e3, 1e3 - 25e3, 0.48 - 0.499, 0.48 - 0.499, 10e - 12-10e10, 10e-12 - 10e10") 
    print(f"Average parameter loss: {np.mean(np.reshape(np.array(parameter_loss), (-1, 6)), axis=0)}")        
    print(f"Average overall loss: {np.sum(losses)/processed}")
    return losses, parameter_loss, val_losses

def test(test_loader, net):
    net.to(DEVICE)
    net.eval()
    criterion = nn.L1Loss()
    crit = nn.L1Loss()
    

    with torch.no_grad():
            loss = 0
            with tqdm(test_loader, unit=" batch") as it:
                for batch in enumerate(it):
                    loss = loss_fn(out, batch.y)
                    loss.backward()
                    optimizer.step()
                    
                    predicted = net(inp) 
                    cost = criterion(out, predicted)
                    l_t = cost.item()
                    loss += l_t
                    for i in range(len(predicted)):
                        p = predicted[i].cpu().numpy().reshape(1, -1)
                        o = out[i].cpu().numpy().reshape(1, -1)
                        print(F"Predicted: {SCALER.inverse_transform(p)}")
                        print(F"Real: {SCALER.inverse_transform(o)}")
                        print(f"Difference: {abs(p - o)}\n\n")
                    
                    #print(f"\n\n\nBatch: {idx}")
                   # print(f"loss: {l_t}")
                    #for i, target in enumerate(out):
                   #     errs = []
                   #     print(f"Targer: {target}, \npredicted: {predicted[i]}\n\n")
                   #     for j in range(len(predicted)):
                   #         errs.append(abs(predicted[i]-target[i])**2)
                   #     print(f"MSE: {np.mean(errs[0])}")
            
            print(f"Average Loss: {loss/len(test_loader)}") 

In [145]:
dataset[7].edge_index.shape

torch.Size([4, 16129])

In [156]:
# Instantiate the dataset
dataset = SignalDataset(runs=runs[:10])

train_n = int(0.8 * len(dataset))
test_n = len(dataset) - train_n
train_set, test_set = random_split(dataset, [train_n, test_n])
train_loader, test_loader = DataLoader(train_set, batch_size=2, shuffle=True, collate_fn=custom_collate), \
                            DataLoader(test_set, batch_size=2, shuffle=True, collate_fn=custom_collate)

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 202.07it/s]


In [157]:
# Train the model
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Initialize the model
model = GCNModel(in_channels=2, hidden_channels=512, out_channels=6)
train(train_loader, model)


Using: cpu


Epoch 1/2000:   0%|                                                                           | 0/4 [00:00<?, ?batch/s]

torch.Size([8]) torch.Size([4, 32258]) torch.Size([4, 128])


IndexError: The shape of the mask [32258] at index 0 does not match the shape of the indexed tensor [8] at index 0

In [ ]:
# Define loss and optimizer
loss_fn = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
for epoch in range(100):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch)
        loss = loss_fn(out, batch.y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        test_loss = 0
        for batch in train_loader:
            out = model(batch)
            test_loss += F.mse_loss(out, batch.y).item() * batch.num_graphs
        test_loss /= len(train_loader.dataset)

    print(f'Epoch: {epoch}, Loss: {test_loss:.4f}')

In [141]:
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data as GData, Batch
from sklearn.preprocessing import MinMaxScaler
from scipy.interpolate import interp1d
import pickle
import torch_geometric.transforms as T
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import torch

SCALER = MinMaxScaler()

class SignalDataset(Dataset):
    def __init__(self, signalFolder="D:/SamplingResults2", sampleFile="newSamples.pkl", runs=range(100), steps=128):
        # Load both disp1 and disp2 from each folder
        # Folders ordered according to index of sample
        # Use the corresponding sample as y -> append probe?
        self.data = []
        self.output = []
        self.input = []
        
        with open(f"{sampleFile}", "rb") as f:
             samples = pickle.load(f)
        
        # Gather all the data first
        inputs = []
        for run in tqdm(runs):
            inp = []
            fail = False
            
            files = os.listdir(f"{signalFolder}/{run}/")
            
            if files != ['Disp1.csv', 'Disp2.csv']:
                continue
            
            for file in files:
                a = pd.read_csv(f"{signalFolder}/{run}/{file}")
                a.rename(columns = {'0':'x', '0.1': 'y'}, inplace = True)
                
                if a['x'].max() != 7.0:
                    fail = True
                    break
                
                # Interpolate curve for consistent x values
                xNew = np.linspace(0, 7, num=steps, endpoint=False)
                interped = interp1d(a['x'], a['y'], kind='cubic', fill_value="extrapolate")(xNew)
                    
                inp.append(interped.astype("float32"))
            
            if not fail:
                if len(inp) != 2:
                    raise Exception("sdf")

                self.input.append(inp)

                self.output.append(samples[int(run)])

        # Scale input data
        
        SCALER.fit_transform(self.output)
        self.output = SCALER.transform(self.output)
        self.output = self.output.reshape(-1, 6)
        self.output = torch.tensor(self.output, dtype=torch.float)
        
        # Add y values to data objects
        for i, input_data in enumerate(self.input):
            # Create node features tensor
            x = torch.tensor(input_data, dtype=torch.float)

            # Create edge_index tensor for the concatenated graph
            num_nodes_1 = len(inp[0])
            num_nodes_2 = len(inp[1])
            edge_index_1 = torch.tensor([[i, j] for i in range(num_nodes_1 - 1) for j in range(num_nodes_1, num_nodes_1 + num_nodes_2 - 1)], dtype=torch.long)
            edge_index_2 = torch.tensor([[j, i] for i, j in edge_index_1], dtype=torch.long)
            edge_index = torch.cat([edge_index_1, edge_index_2], dim=-1)

            # Set edge weights to 1 for all adjacent edges, and negative values for edges connecting nodes from different graphs
            edge_weight = torch.ones(edge_index.shape[1])
            edge_weight[num_nodes_1-1::num_nodes_1] = -1.
            self.data.append(Data(x=x, edge_index=edge_index.t().contiguous(), edge_weight=edge_weight, y=self.output[i]))        

        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def custom_collate(batch):
    """
    Collate function that can handle PyTorch Geometric Data objects.
    """
    return Batch.from_data_list(batch)


In [9]:
with open("filtered.pkl", "rb") as f:
    runs = pickle.load(f)